# Question revision chain

- get a question
- LLM A and B verify if the question is OK
    - there is more than one option correct
    - sugests modifications to improve the questions and options
    - sugest a detailed explanation to give to the student
- verify 
    - if they diverge, calls another LLM to solve vote
    - if ok, merge using OpenAI

In [6]:
from dotenv import load_dotenv
import json_repair
import os
import pandas as pd

In [7]:
df = pd.read_csv("questions.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 953 non-null    int64 
 1   created_at         953 non-null    object
 2   subject_matter     953 non-null    object
 3   topic_description  953 non-null    object
 4   level              953 non-null    object
 5   question           953 non-null    object
 6   type               953 non-null    object
 7   answer_correct     953 non-null    object
 8   explanation        953 non-null    object
 9   answer_a           953 non-null    object
 10  answer_b           953 non-null    object
 11  answer_c           953 non-null    object
 12  answer_d           953 non-null    object
dtypes: int64(1), object(12)
memory usage: 96.9+ KB


In [2]:
import google.generativeai as genai

key_gemini = os.environ["GOOGLE_API_KEY"]

genai.configure(api_key=key_gemini)

/workspaces/quiz_interview/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Create the model
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    # "top_k": 64,
    # "max_output_tokens": 8192,
    "response_mime_type": "application/json",
}

llm_gemini = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
    # safety_settings = Adjust safety settings
    # See https://ai.google.dev/gemini-api/docs/safety-settings
)

In [5]:
# llm_gemini.generate_content("quem é vc?")

In [ ]:
prompt_revision = (
    "You are an expert in machine learning."
    "your task is to analyse the provided question to verify if there is some problem."
    "You shoud verify if there is just one option correct or if there is more than one."
    "Also, you should give a detailed explanation for your answer."
    "# QUESTION"
    "{question}\n\n"
    # "# OUTPUT FORMAT\n"
    # "{output_format}"
)